In [1]:
import sys
print(sys.executable)
print(sys.version)

c:\Users\juand\Documents\Universidad Septimo Semestre\Procesamiento Digital de Imagenes\Colab\.venv\Scripts\python.exe
3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]
